In [ ]:
%load_ext autoreload
%autoreload 2

import json
from dwitracts.glm import DwiTractsGlm
import dwitracts.plot as dwiplot

In [ ]:
# GLM Analysis using DwiTractsGlm class

config_file = 'project/config_glm_dmn.json'

def _json_object_hook(d): return namedtuple('X', d.keys())(*d.values())
def json2obj(data): return json.loads(data, object_hook=_json_object_hook)

with open(config_file, 'r') as myfile:
    json_string=myfile.read()
    
params = json.loads(json_string)
params_gen = params['general']
params_glm = params['glm']

with open(params_gen['tracts_config_file'], 'r') as myfile:
    json_string=myfile.read()
    
params['tracts'] = json.loads(json_string)

with open(params_gen['preproc_config_file'], 'r') as myfile:
    json_string=myfile.read()
    
params['preproc'] = json.loads(json_string)

my_glm = DwiTractsGlm( params )

# Initialize the object with the current parameters
assert my_glm.initialize( )


In [ ]:
# Fit all specified GLMs and save results to target directory
assert my_glm.fit_glms( clobber=True, verbose=True )


In [ ]:
# Extract distance traces from the GLM results
assert my_glm.extract_distance_traces( clobber=True, verbose=True )


In [ ]:
# Extract distance traces from the GLM results, using RFT1D cluster inference
assert my_glm.extract_distance_traces_rft1d( clobber=True, verbose=True, debug=True )



In [ ]:
# Extract distance traces from the GLM results, using cluster inference and permutation testing
# my_glm.extract_distance_trace_clusters( clobber=True, verbose=True, debug=True )


In [ ]:
# Plot t-value distance traces

# Font sizes
params['axis_font'] = 24
params['yticklabel_font'] = 16
params['xticklabel_font'] = 20
params['title_font'] = 40
params['font_scale'] = 1.5

alpha = 0.05

tract_names = my_glm.tract_names.copy()
tract_names.sort()

# Uncorrected
dwiplot.plot_distance_traces( params, tract_names, alpha=alpha, stat_type='uncorrected' )

# RFT corrected
dwiplot.plot_distance_traces( params, tract_names, alpha=alpha, stat_type='rft' )

# Permutation corrected
# dwiplot.plot_distance_traces( params, tract_names, alpha=alpha, stat_type='perm' )


In [ ]:
# Plot scatterplots & violin plots for GLM results

# Specify GLM
glm = 'glm-age-sex'

# Set up parameters
params_plot = {}
params_plot['glm'] = glm
params_plot['alpha'] = 0.05
params_plot['outlier_z'] = params_glm[glm]['outlier_z']
params_plot['stat_type'] = 'rft'
params_plot['axis_font'] = 40
params_plot['marker_size'] = 15
params_plot['line_width'] = 4
params_plot['ticklabel_font'] = 33
params_plot['title_font'] = 45
params_plot['legend_font'] = 40
params_plot['legend_scale'] = 3
params_plot['show_title'] = True
params_plot['show_legend'] = True
params_plot['dimensions'] = (12,10)
params_plot['font_scale'] = 0.6
params_plot['write_csv'] = False

# Loop through tracts and call plot function for each
dwiplot.plot_glm_results_all( params_plot, my_glm, verbose=True )
    

In [ ]:
# Aggregate tracts into single volumes
assert my_glm.aggregate_tracts(prefix='tract_final_norm_bidir', op='max', verbose=True)


In [ ]:
# Aggregate tracts crossing corpus callosum
cc_img = '/Volumes/GDriveUsb/data/atlases/corpus_callosum_midline.nii'
assert my_glm.aggregate_tracts(prefix='tract_final_norm_bidir', suffix='_cc', intersects=cc_img, \
                                op='max', threshold=0, verbose=True)


In [ ]:
# Aggregate stats into single volumes

assert my_glm.aggregate_stats( op='max', pthres=0.05, suffix='-rft', verbose=True, clobber=True )


In [ ]:
# Generate Pajek graphs

assert my_glm.create_pajek_graphs( suffix='-rft', edge_val='tsum', verbose=True, clobber=True )
